In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import joblib

import warnings
from sklearn.preprocessing import OneHotEncoder, StandardScaler

warnings.filterwarnings('ignore')

In [2]:
#Importation du jeu de donnée
pd.set_option('display.max_columns', None)
pd.set_option('display.max_row', None)

# Pour charger le DataFrame à partir du fichier Joblib
df_cluster_sample = joblib.load('df_cluster_sample.joblib')

rfm_data = joblib.load('rfm_data.joblib')

In [3]:
df_cluster_sample.shape

(23010, 36)

In [4]:
list(df_cluster_sample)

['order_id',
 'order_item_id',
 'product_id',
 'seller_id',
 'price',
 'freight_value',
 'product_category_name',
 'payment_type',
 'payment_installments',
 'payment_value',
 'customer_id',
 'order_approved_at',
 'customer_unique_id',
 'customer_zip_code_prefix',
 'customer_city',
 'customer_state',
 'review_score',
 'seller_zip_code_prefix',
 'seller_city',
 'seller_state',
 'customer_geo_lat',
 'customer_geo_lng',
 'seller_geo_lat',
 'seller_geo_lng',
 'time_between_orders_same_customer',
 'days_between_orders_same_customer',
 'hours_between_orders_same_customer',
 'time_between_orders_all',
 'days_between_orders_all',
 'hours_between_orders_all',
 'order_month',
 'day_of_week',
 'day_of_month',
 'distance',
 'cluster',
 'Recency']

In [ ]:
from datetime import datetime, timedelta
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import numpy as np
import pandas as pd

# Charger le DataFrame initial (en supposant que vous avez un DataFrame nommé df_cluster_sample)
df = df_cluster_sample.copy()

# Ajouter une colonne 'year' basée sur 'order_approved_at'
df['year'] = df['order_approved_at'].dt.year

# Filtrer les données pour les années 2017 et 2018
df = df[(df['year'] == 2017) | (df['year'] == 2018)]
# Colonnes à supprimer
colonnes_a_supprimer = ['seller_zip_code_prefix', 'seller_city', 'seller_state', 'customer_geo_lat', 'customer_geo_lng',
                   'seller_geo_lat', 'seller_geo_lng', 'time_between_orders_same_customer',
                   'days_between_orders_same_customer', 'hours_between_orders_same_customer',
                   'time_between_orders_all', 'days_between_orders_all', 'hours_between_orders_all',
                   'order_id', 'product_id', 'seller_id', 'customer_id', 'customer_unique_id']

# Supprimer les colonnes spécifiées
df = df.drop(columns=colonnes_a_supprimer)

# Fonction pour créer un ensemble de données jusqu'à une date donnée
def creer_ensemble_de_donnees(df, date_fin, scaler, encodeur, noms_caracteristiques_de_base):
    ensemble_de_donnees = df[df['order_approved_at'] <= date_fin].drop('order_approved_at', axis=1)
    
    # Séparer les colonnes numériques et catégorielles
    colonnes_numeriques = ensemble_de_donnees.select_dtypes(include=np.number).columns
    colonnes_catégorielles = ensemble_de_donnees.select_dtypes(exclude=np.number).columns

    # Ajuster et transformer les colonnes numériques
    ensemble_de_donnees[colonnes_numeriques] = scaler.fit_transform(ensemble_de_donnees[colonnes_numeriques])

    # Transformer les colonnes catégorielles et créer un nouveau DataFrame
    cat_transformed = pd.DataFrame(encodeur.fit_transform(ensemble_de_donnees[colonnes_catégorielles]),
                                   columns=encodeur.get_feature_names_out(colonnes_catégorielles), index=ensemble_de_donnees.index)

    # Concaténer les colonnes numériques et catégorielles transformées
    ensemble_de_donnees = pd.concat([ensemble_de_donnees[colonnes_numeriques], cat_transformed], axis=1)

    # Assurer la cohérence dans les noms des caractéristiques
    ensemble_de_donnees.columns = ensemble_de_donnees.columns.astype(str)

    # Obtenir les caractéristiques manquantes des noms de caractéristiques de base
    caractéristiques_manquantes = set(noms_caracteristiques_de_base) - set(ensemble_de_donnees.columns)

    # Ajouter les caractéristiques manquantes avec des zéros comme valeurs
    for caractéristique in caractéristiques_manquantes:
        ensemble_de_donnees[caractéristique] = 0

    # Réorganiser les colonnes pour correspondre aux noms de caractéristiques de base
    ensemble_de_donnees = ensemble_de_donnees[noms_caracteristiques_de_base]

    return ensemble_de_donnees

# Entraînement initial du modèle sur F0 (en supposant que T0 est le début du jeu de données)
scaler_m0 = StandardScaler()
encodeur_m0 = OneHotEncoder(sparse=False)

# Ajuster encodeur_m0 sur l'ensemble du jeu de données pour obtenir les noms de caractéristiques
colonnes_catégorielles = df.select_dtypes(exclude=np.number).columns
encodeur_m0.fit(df[colonnes_catégorielles])

# Initialize meilleur_ari before the loop
meilleur_ari = 0

F0 = creer_ensemble_de_donnees(df, df['order_approved_at'].min() + timedelta(weeks=2), scaler_m0, encodeur_m0, df.columns)  # 2 semaines à partir de la date de début

# Convertir tous les noms de colonnes en chaînes
F0.columns = F0.columns.astype(str)

# Initialiser les modèles M0, M1, M2 en dehors de la boucle
M0 = KMeans(n_clusters=6)
M1 = KMeans(n_clusters=6)
M2 = KMeans(n_clusters=6)

# Ajuster M0 sur F0 (ensemble de données initial)
M0.fit(F0)
c0_m0 = M0.predict(F0)

# Initialiser les flags pour réentrainer M0, M1, M2
retrain_m0 = False
retrain_m1 = False
retrain_m2 = False

# Initialiser la date_actuelle et jours_entre_les_simulations
date_actuelle = df['order_approved_at'].min() + timedelta(weeks=2)
jours_entre_les_simulations = 14

while date_actuelle <= df['order_approved_at'].max():
    # Créer l'ensemble de données Fi
    Fi = creer_ensemble_de_donnees(df, date_actuelle, scaler_m0, encodeur_m0, df.columns)
    
    # Convertir tous les noms de colonnes en chaînes
    Fi.columns = Fi.columns.astype(str)

    # Entraîner un nouveau modèle Mi sur Fi
    scaler_mi = StandardScaler()
    encodeur_mi = OneHotEncoder(sparse=False)

    # Utiliser les noms de caractéristiques de l'encodeur_m0 pour assurer la cohérence
    encodeur_mi.fit(df[colonnes_catégorielles])

    Fi_mise_a_l_echelle = creer_ensemble_de_donnees(df, date_actuelle, scaler_mi, encodeur_mi, df.columns)

    # Assurer la cohérence dans les noms des caractéristiques
    Fi_mise_a_l_echelle.columns = Fi_mise_a_l_echelle.columns.astype(str)

    # Obtenir les caractéristiques manquantes du modèle initial
    caractéristiques_manquantes = set(F0.columns) - set(Fi_mise_a_l_echelle.columns)

    # Ajouter les caractéristiques manquantes avec des zéros comme valeurs
    for caractéristique in caractéristiques_manquantes:
        Fi_mise_a_l_echelle[caractéristique] = 0

    # Réorganiser les colonnes pour correspondre au modèle initial
    Fi_mise_a_l_echelle = Fi_mise_a_l_echelle[F0.columns]

    Mi = KMeans(n_clusters=5)
    Mi.fit(Fi_mise_a_l_echelle)
    c1_mi = Mi.predict(Fi_mise_a_l_echelle)

    # Prédire les clusters de Fi en utilisant le modèle initial M0
    c1_m0 = M0.predict(Fi_mise_a_l_echelle)

    # Calculer l'ARI
    ari = adjusted_rand_score(c1_mi, c1_m0)
    print(f"ARI à {date_actuelle}: {ari}")

    # Mettre à jour le meilleur ARI et sa date si nécessaire
    if ari > meilleur_ari:
        meilleur_ari = ari
        meilleure_date = date_actuelle

    # Déterminer si le modèle M0 devient obsolète
    if ari < 0.8:
        print(f"Le modèle M0 devient obsolète à {date_actuelle}")
        F0_mise_a_l_echelle = creer_ensemble_de_donnees(df, date_actuelle, scaler_m0, encodeur_m0, df.columns)
        
        # Convertir tous les noms de colonnes en chaînes
        F0_mise_a_l_echelle.columns = F0_mise_a_l_echelle.columns.astype(str)
        
        M0.fit(F0_mise_a_l_echelle)
        print("Le modèle M0 a été réentraîné.")
        # Réentraîner le modèle M1 dès le jour suivant
        retrain_m1 = True

    # Réentraîner le modèle M1 dès le jour suivant si nécessaire
    if retrain_m1:
        print(f"Réentraînement de M1 à {date_actuelle + timedelta(days=1)}")
        Fi_mise_a_l_echelle = creer_ensemble_de_donnees(df, date_actuelle + timedelta(days=1), scaler_m0, encodeur_m0, df.columns)
        
        # Convertir tous les noms de colonnes en chaînes
        Fi_mise_a_l_echelle.columns = Fi_mise_a_l_echelle.columns.astype(str)
        
        M1.fit(Fi_mise_a_l_echelle)
        print("Le modèle M1 a été réentraîné.")
        # Réentraîner le modèle M2 dès le jour suivant
        retrain_m2 = True

    # Réentraîner le modèle M2 dès le jour suivant si nécessaire
    if retrain_m2:
        print(f"Réentraînement de M2 à {date_actuelle + timedelta(days=1)}")
        Fi_mise_a_l_echelle = creer_ensemble_de_donnees(df, date_actuelle + timedelta(days=1), scaler_m0, encodeur_m0, df.columns)
        
        # Convertir tous les noms de colonnes en chaînes
        Fi_mise_a_l_echelle.columns = Fi_mise_a_l_echelle.columns.astype(str)
        
        M2.fit(Fi_mise_a_l_echelle)
        print("Le modèle M2 a été réentraîné.")

    # Passer à la période suivante
    date_actuelle += timedelta(days=jours_entre_les_simulations)

# Afficher le meilleur ARI et sa date correspondante après la boucle
print(f"Meilleur ARI : {meilleur_ari} à {meilleure_date}")


ARI à 2017-01-19 23:05:27: 0.8449112242215691
ARI à 2017-02-02 23:05:27: 0.7670035897802935
Le modèle M0 devient obsolète à 2017-02-02 23:05:27
Le modèle M0 a été réentraîné.
Réentraînement de M1 à 2017-02-03 23:05:27
Le modèle M1 a été réentraîné.
Réentraînement de M2 à 2017-02-03 23:05:27
Le modèle M2 a été réentraîné.
ARI à 2017-02-16 23:05:27: 0.4241338565277158
Le modèle M0 devient obsolète à 2017-02-16 23:05:27
Le modèle M0 a été réentraîné.
Réentraînement de M1 à 2017-02-17 23:05:27
Le modèle M1 a été réentraîné.
Réentraînement de M2 à 2017-02-17 23:05:27
Le modèle M2 a été réentraîné.
ARI à 2017-03-02 23:05:27: 0.4632685453059713
Le modèle M0 devient obsolète à 2017-03-02 23:05:27
Le modèle M0 a été réentraîné.
Réentraînement de M1 à 2017-03-03 23:05:27
Le modèle M1 a été réentraîné.
Réentraînement de M2 à 2017-03-03 23:05:27
Le modèle M2 a été réentraîné.
ARI à 2017-03-16 23:05:27: 0.4103529443188131
Le modèle M0 devient obsolète à 2017-03-16 23:05:27
Le modèle M0 a été réentr

In [5]:
from datetime import datetime, timedelta
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import numpy as np
import pandas as pd

def create_dataset(df, end_date, scaler, encoder, base_feature_names):
    """
    Crée un ensemble de données jusqu'à une date donnée.
    
    Paramètres:
    - df: DataFrame, DataFrame d'entrée.
    - end_date: datetime, La date de fin.
    - scaler: StandardScaler, le scaler pour les colonnes numériques.
    - encoder: OneHotEncoder, l'encodeur pour les colonnes catégorielles.
    - base_feature_names: liste, noms des caractéristiques de base.

    Retourne:
    DataFrame, l'ensemble de donnée créé.
    """
    dataset = df[df['order_approved_at'] <= end_date].drop('order_approved_at', axis=1)

    # Separate numeric and categorical columns
    numeric_columns = dataset.select_dtypes(include=np.number).columns
    categorical_columns = dataset.select_dtypes(exclude=np.number).columns

    # Adjust and transform numeric columns
    dataset[numeric_columns] = scaler.fit_transform(dataset[numeric_columns])

    # Transform categorical columns and create a new DataFrame
    cat_transformed = pd.DataFrame(encoder.fit_transform(dataset[categorical_columns]),
                                   columns=encoder.get_feature_names_out(categorical_columns), index=dataset.index)

    # Concatenate transformed numeric and categorical columns
    dataset = pd.concat([dataset[numeric_columns], cat_transformed], axis=1)

    # Ensure consistency in feature names 
    dataset.columns = dataset.columns.astype(str)

    # Get missing features from base feature names
    missing_features = set(base_feature_names) - set(dataset.columns)

    # Add missing features with zero values
    for feature in missing_features:
        dataset[feature] = 0

    # Reorder columns to match base feature names
    dataset = dataset[base_feature_names]

    return dataset


# Load the initial DataFrame (assuming you have a DataFrame named df_cluster_sample)
df = df_cluster_sample.copy()

# Add a 'year' column based on 'order_approved_at'
df['year'] = df['order_approved_at'].dt.year

# Filter data for the years 2017 and 2018
df = df[(df['year'] == 2017) | (df['year'] == 2018)]

# Columns to delete
columns_to_delete = ['seller_zip_code_prefix', 'seller_city', 'seller_state', 'customer_geo_lat', 'customer_geo_lng',
                   'seller_geo_lat', 'seller_geo_lng', 'time_between_orders_same_customer',
                   'days_between_orders_same_customer', 'hours_between_orders_same_customer',
                   'time_between_orders_all', 'days_between_orders_all', 'hours_between_orders_all',
                   'order_id', 'product_id', 'seller_id', 'customer_id', 'customer_unique_id']

# Delete specified columns
df = df.drop(columns=columns_to_delete)

# Initial training of the model on F0 (assuming T0 is the start of the dataset)
scaler_m0 = StandardScaler()
encoder_m0 = OneHotEncoder(sparse=False)

# Adjust encoder_m0 on the entire dataset to get feature names
categorical_columns = df.select_dtypes(exclude=np.number).columns
encoder_m0.fit(df[categorical_columns])

# Initialize best_ari before the loop
best_ari = 0

F0 = create_dataset(df, df['order_approved_at'].min() + timedelta(weeks=2), scaler_m0, encoder_m0, df.columns)

# Convert all column names to strings
F0.columns = F0.columns.astype(str)

# Initialize models M0, M1, M2 outside the loop
M0 = KMeans(n_clusters=6)
M1 = KMeans(n_clusters=6)
M2 = KMeans(n_clusters=6)

# Train M0 on F0 (initial dataset)
M0.fit(F0)
c0_m0 = M0.predict(F0)

# Initialize flags to retrain M0, M1, M2
retrain_m0 = False
retrain_m1 = False
retrain_m2 = False

# Initialize current_date and days_between_simulations
current_date = df['order_approved_at'].min() + timedelta(weeks=2)
days_between_simulations = 14

while current_date <= df['order_approved_at'].max():
    # Create dataset Fi
    Fi = create_dataset(df, current_date, scaler_m0, encoder_m0, df.columns)

    # Convert all column names to strings
    Fi.columns = Fi.columns.astype(str)

    # Train a new model Mi on Fi
    scaler_mi = StandardScaler()
    encoder_mi = OneHotEncoder(sparse=False)

    # Use encoder_m0 feature names for consistency
    encoder_mi.fit(df[categorical_columns])

    Fi_scaled = create_dataset(df, current_date, scaler_mi, encoder_mi, df.columns)

    # Ensure consistency in feature names
    Fi_scaled.columns = Fi_scaled.columns.astype(str)

    # Get missing features from the initial model
    missing_features = set(F0.columns) - set(Fi_scaled.columns)

    # Add missing features with zero values
    for feature in missing_features:
        Fi_scaled[feature] = 0

    # Reorder columns to match the initial model
    Fi_scaled = Fi_scaled[F0.columns]

    Mi = KMeans(n_clusters=5)
    Mi.fit(Fi_scaled)
    c1_mi = Mi.predict(Fi_scaled)

    # Predict clusters of Fi using the initial model M0
    c1_m0 = M0.predict(Fi_scaled)

    # Calculate adjusted_rand_score (ARI)
    ari = adjusted_rand_score(c1_mi, c1_m0)
    print(f"ARI at {current_date}: {ari}")

    # Update the best ARI and its date if necessary
    if ari > best_ari:
        best_ari = ari
        best_date = current_date

    # Check if the model M0 becomes obsolete
    if ari < 0.8:
        print(f"M0 becomes obsolete at {current_date}")
        F0_scaled = create_dataset(df, current_date, scaler_m0, encoder_m0, df.columns)

        # Convert all column names to strings
        F0_scaled.columns = F0_scaled.columns.astype(str)

        M0.fit(F0_scaled)
        print("M0 has been retrained.")
        # Retrain model M1 from the next day
        retrain_m1 = True

    # Retrain model M1 from the next day if necessary
    if retrain_m1:
        print(f"Retraining M1 at {current_date + timedelta(days=1)}")
        Fi_scaled = create_dataset(df, current_date + timedelta(days=1), scaler_m0, encoder_m0, df.columns)

        # Convert all column names to strings
        Fi_scaled.columns = Fi_scaled.columns.astype(str)

        M1.fit(Fi_scaled)
        print("M1 has been retrained.")
        # Retrain model M2 from the next day
        retrain_m2 = True

    # Retrain model M2 from the next day if necessary
    if retrain_m2:
        print(f"Retraining M2 at {current_date + timedelta(days=1)}")
        Fi_scaled = create_dataset(df, current_date + timedelta(days=1), scaler_m0, encoder_m0, df.columns)

        # Convert all column names to strings
        Fi_scaled.columns = Fi_scaled.columns.astype(str)

        M2.fit(Fi_scaled)
        print("M2 has been retrained.")

    # Move to the next period
    current_date += timedelta(days=days_between_simulations)

# Display the best ARI and its corresponding date after the loop
print(f"Best ARI: {best_ari} at {best_date}")


ARI at 2017-01-19 23:05:27: 0.8734704175602371
ARI at 2017-02-02 23:05:27: 0.508866471473944
M0 becomes obsolete at 2017-02-02 23:05:27
M0 has been retrained.
Retraining M1 at 2017-02-03 23:05:27
M1 has been retrained.
Retraining M2 at 2017-02-03 23:05:27
M2 has been retrained.
ARI at 2017-02-16 23:05:27: 0.4018665588958842
M0 becomes obsolete at 2017-02-16 23:05:27
M0 has been retrained.
Retraining M1 at 2017-02-17 23:05:27
M1 has been retrained.
Retraining M2 at 2017-02-17 23:05:27
M2 has been retrained.
ARI at 2017-03-02 23:05:27: 0.5582824898944708
M0 becomes obsolete at 2017-03-02 23:05:27
M0 has been retrained.
Retraining M1 at 2017-03-03 23:05:27
M1 has been retrained.
Retraining M2 at 2017-03-03 23:05:27
M2 has been retrained.
ARI at 2017-03-16 23:05:27: 0.4525065796278576
M0 becomes obsolete at 2017-03-16 23:05:27
M0 has been retrained.
Retraining M1 at 2017-03-17 23:05:27
M1 has been retrained.
Retraining M2 at 2017-03-17 23:05:27
M2 has been retrained.
ARI at 2017-03-30 23:0


KeyboardInterrupt



In [4]:
from datetime import datetime, timedelta
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import pandas as pd
import numpy as np

# Charger le DataFrame initial (en supposant que vous avez un DataFrame nommé df_cluster_sample)
df = df_cluster_sample.copy()

# Colonnes à supprimer
colonnes_a_supprimer = ['seller_zip_code_prefix', 'seller_city', 'seller_state', 'customer_geo_lat', 'customer_geo_lng',
                   'seller_geo_lat', 'seller_geo_lng', 'time_between_orders_same_customer',
                   'days_between_orders_same_customer', 'hours_between_orders_same_customer',
                   'time_between_orders_all', 'days_between_orders_all', 'hours_between_orders_all',
                   'order_id', 'product_id', 'seller_id', 'customer_id', 'customer_unique_id']

# Supprimer les colonnes spécifiées
df = df.drop(columns=colonnes_a_supprimer)

# Fonction pour créer un ensemble de données jusqu'à une date donnée
def creer_ensemble_de_donnees(df, date_fin, scaler, encodeur, noms_caracteristiques_de_base):
    ensemble_de_donnees = df[df['order_approved_at'] <= date_fin].drop('order_approved_at', axis=1)
    
    # Séparer les colonnes numériques et catégorielles
    colonnes_numeriques = ensemble_de_donnees.select_dtypes(include=np.number).columns
    colonnes_catégorielles = ensemble_de_donnees.select_dtypes(exclude=np.number).columns

    # Ajuster et transformer les colonnes numériques
    ensemble_de_donnees[colonnes_numeriques] = scaler.fit_transform(ensemble_de_donnees[colonnes_numeriques])

    # Transformer les colonnes catégorielles et créer un nouveau DataFrame
    cat_transformed = pd.DataFrame(encodeur.fit_transform(ensemble_de_donnees[colonnes_catégorielles]),
                                   columns=encodeur.get_feature_names_out(colonnes_catégorielles), index=ensemble_de_donnees.index)

    # Concaténer les colonnes numériques et catégorielles transformées
    ensemble_de_donnees = pd.concat([ensemble_de_donnees[colonnes_numeriques], cat_transformed], axis=1)

    # Assurer la cohérence dans les noms des caractéristiques
    ensemble_de_donnees.columns = ensemble_de_donnees.columns.astype(str)

    # Obtenir les caractéristiques manquantes des noms de caractéristiques de base
    caractéristiques_manquantes = set(noms_caracteristiques_de_base) - set(ensemble_de_donnees.columns)

    # Ajouter les caractéristiques manquantes avec des zéros comme valeurs
    for caractéristique in caractéristiques_manquantes:
        ensemble_de_donnees[caractéristique] = 0

    # Réorganiser les colonnes pour correspondre aux noms de caractéristiques de base
    ensemble_de_donnees = ensemble_de_donnees[noms_caracteristiques_de_base]

    return ensemble_de_donnees

# Entraînement initial du modèle sur F0 (en supposant que T0 est le début du jeu de données)
scaler_m0 = StandardScaler()
encodeur_m0 = OneHotEncoder(sparse=False)

# Ajuster encodeur_m0 sur l'ensemble du jeu de données pour obtenir les noms de caractéristiques
colonnes_catégorielles = df.select_dtypes(exclude=np.number).columns
encodeur_m0.fit(df[colonnes_catégorielles])

F0 = creer_ensemble_de_donnees(df, df['order_approved_at'].min() + timedelta(weeks=2), scaler_m0, encodeur_m0, df.columns)  # 2 semaines à partir de la date de début

# Convertir tous les noms de colonnes en chaînes
F0.columns = F0.columns.astype(str)

m0 = KMeans(n_clusters=5)
m0.fit(F0)
c0_m0 = m0.predict(F0)

# Itération à travers les périodes
jours_entre_les_simulations = 14
date_actuelle = df['order_approved_at'].min() + timedelta(weeks=2)

# Initialiser les variables pour stocker le meilleur ARI, sa date correspondante, et la durée d'obsolescence
meilleur_ari = 0
meilleure_date = None
duree_obsolescence = 0

# Initialiser les modèles M1, M2
M1 = KMeans(n_clusters=5)
M2 = KMeans(n_clusters=5)

while date_actuelle <= df['order_approved_at'].max():
    # Créer l'ensemble de données Fi
    Fi = creer_ensemble_de_donnees(df, date_actuelle, scaler_m0, encodeur_m0, df.columns)
    
    # Convertir tous les noms de colonnes en chaînes
    Fi.columns = Fi.columns.astype(str)

    # Entraîner un nouveau modèle Mi sur Fi
    scaler_mi = StandardScaler()
    encodeur_mi = OneHotEncoder(sparse=False)

    # Utiliser les noms de caractéristiques de l'encodeur_m0 pour assurer la cohérence
    encodeur_mi.fit(df[colonnes_catégorielles])

    Fi_mise_a_l_echelle = creer_ensemble_de_donnees(df, date_actuelle, scaler_mi, encodeur_mi, df.columns)

    # Assurer la cohérence dans les noms des caractéristiques
    Fi_mise_a_l_echelle.columns = Fi_mise_a_l_echelle.columns.astype(str)

    # Obtenir les caractéristiques manquantes du modèle initial
    caractéristiques_manquantes = set(F0.columns) - set(Fi_mise_a_l_echelle.columns)

    # Ajouter les caractéristiques manquantes avec des zéros comme valeurs
    for caractéristique in caractéristiques_manquantes:
        Fi_mise_a_l_echelle[caractéristique] = 0

    # Réorganiser les colonnes pour correspondre au modèle initial
    Fi_mise_a_l_echelle = Fi_mise_a_l_echelle[F0.columns]

    # Utiliser le modèle M0
    c1_m0 = m0.predict(Fi_mise_a_l_echelle)

    # Sélectionner le modèle en fonction de la durée d'obsolescence
    if duree_obsolescence == 0:
        Mi = M1
    elif duree_obsolescence == 14:
        Mi = M2
    # Ajouter des conditions pour d'autres modèles si nécessaire

    # Entraîner le modèle Mi sur Fi
    Mi.fit(Fi_mise_a_l_echelle)
    c1_mi = Mi.predict(Fi_mise_a_l_echelle)

    # Calculer l'ARI
    ari = adjusted_rand_score(c1_mi, c1_m0)
    print(f"ARI à {date_actuelle}: {ari}")

    # Mettre à jour le meilleur ARI, sa date et la durée d'obsolescence si nécessaire
    if ari > meilleur_ari:
        meilleur_ari = ari
        meilleure_date = date_actuelle

    # Déterminer si le modèle a besoin d'être réentraîné
    if ari < 0.8:
        print(f"Le modèle M0 devient obsolète à {date_actuelle}")
        duree_obsolescence += jours_entre_les_simulations  # Ajouter la durée d'obsolescence
        F0_mise_a_l_echelle = creer_ensemble_de_donnees(df, date_actuelle, scaler_m0, encodeur_m0, df.columns)
        
        # Convertir tous les noms de colonnes en chaînes
        F0_mise_a_l_echelle.columns = F0_mise_a_l_echelle.columns.astype(str)
        
        m0.fit(F0_mise_a_l_echelle)
        c0_m0 = m0.predict(F0_mise_a_l_echelle)
        print(f"Le modèle M0 a été réentraîné. Nouvelle durée d'obsolescence : {duree_obsolescence} jours.")

    # Passer à la période suivante
    date_actuelle += timedelta(days=jours_entre_les_simulations)

    # Afficher la durée d'obsolescence si elle existe
    if duree_obsolescence > 0 and (duree_obsolescence % 14 == 0 or date_actuelle > df['order_approved_at'].max()):
        print(f"Durée d'obsolescence actuelle : {duree_obsolescence} jours")

# Afficher le meilleur ARI, sa date correspondante, et la durée d'obsolescence après la boucle
print(f"Meilleur ARI : {meilleur_ari} à {meilleure_date}")
print(f"Durée d'obsolescence totale : {duree_obsolescence} jours")


ARI à 2016-10-18 09:43:32: 0.4067109206654426
Le modèle M0 devient obsolète à 2016-10-18 09:43:32
Le modèle M0 a été réentraîné. Nouvelle durée d'obsolescence : 14 jours.
Durée d'obsolescence actuelle : 14 jours
ARI à 2016-11-01 09:43:32: 0.6389688469895931
Le modèle M0 devient obsolète à 2016-11-01 09:43:32
Le modèle M0 a été réentraîné. Nouvelle durée d'obsolescence : 28 jours.
Durée d'obsolescence actuelle : 28 jours
ARI à 2016-11-15 09:43:32: 0.6763564333097826
Le modèle M0 devient obsolète à 2016-11-15 09:43:32
Le modèle M0 a été réentraîné. Nouvelle durée d'obsolescence : 42 jours.
Durée d'obsolescence actuelle : 42 jours
ARI à 2016-11-29 09:43:32: 0.9786997645460459
Durée d'obsolescence actuelle : 42 jours
ARI à 2016-12-13 09:43:32: 0.9025067191315194
Durée d'obsolescence actuelle : 42 jours
ARI à 2016-12-27 09:43:32: 0.48225094061835433
Le modèle M0 devient obsolète à 2016-12-27 09:43:32
Le modèle M0 a été réentraîné. Nouvelle durée d'obsolescence : 56 jours.
Durée d'obsolescen